In [0]:
#####----INTERN NAME : Siddhaarth S --------#######
#####----VAL. ACCURRACY: 87.25% ------------#######


# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 256
num_classes = 10
epochs = 50
l = 12
num_filter = 18
compression = 1.0
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l =12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [24]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 24)   648         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 24)   96          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 24)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [27]:
# Train the model
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 353s 7ms/step - loss: 1.4890 - acc: 0.4504 - val_loss: 1.7236 - val_acc: 0.4102

Epoch 00001: val_acc improved from -inf to 0.41020, saving model to weights.best.hdf5
Epoch 2/50
48640/50000 [============================>.] - ETA: 8s - loss: 1.0349 - acc: 0.6272

50000/50000 [==============================] - 329s 7ms/step - loss: 1.0320 - acc: 0.6284 - val_loss: 1.3420 - val_acc: 0.5728

Epoch 00002: val_acc improved from 0.41020 to 0.57280, saving model to weights.best.hdf5
Epoch 3/50
50000/50000 [==============================] - 329s 7ms/step - loss: 0.8350 - acc: 0.7025 - val_loss: 1.1301 - val_acc: 0.6535

Epoch 00003: val_acc improved from 0.57280 to 0.65350, saving model to weights.best.hdf5
Epoch 4/50
17152/50000 [=========>....................] - ETA: 3:19 - loss: 0.7157 - acc: 0.7463

50000/50000 [==============================] - 329s 7ms/step - loss: 0.7177 - acc: 0.7455 - val_loss: 1.5197 - val_acc: 0.6154

Epoch 00004: val_acc did not improve from 0.65350
Epoch 5/50
50000/50000 [==============================] - 329s 7ms/step - loss: 0.6266 - acc: 0.7775 - val_loss: 0.9711 - val_acc: 0.7128

Epoch 00005: val_acc improved from 0.65350 to 0.71280, saving model to weights.best.hdf5
Epoch 6/50
17408/50000 [=========>....................] - ETA: 3:18 - loss: 0.5684 - acc: 0.7986

50000/50000 [==============================] - 328s 7ms/step - loss: 0.5680 - acc: 0.7992 - val_loss: 0.7324 - val_acc: 0.7669

Epoch 00006: val_acc improved from 0.71280 to 0.76690, saving model to weights.best.hdf5
Epoch 7/50
50000/50000 [==============================] - 328s 7ms/step - loss: 0.5156 - acc: 0.8189 - val_loss: 0.6908 - val_acc: 0.7803

Epoch 00007: val_acc improved from 0.76690 to 0.78030, saving model to weights.best.hdf5
Epoch 8/50
12288/50000 [======>.......................] - ETA: 3:48 - loss: 0.4699 - acc: 0.8302

50000/50000 [==============================] - 328s 7ms/step - loss: 0.4741 - acc: 0.8322 - val_loss: 0.9445 - val_acc: 0.7314

Epoch 00008: val_acc did not improve from 0.78030
Epoch 9/50
50000/50000 [==============================] - 329s 7ms/step - loss: 0.4443 - acc: 0.8461 - val_loss: 0.6895 - val_acc: 0.7909

Epoch 00009: val_acc improved from 0.78030 to 0.79090, saving model to weights.best.hdf5
Epoch 10/50
16384/50000 [========>.....................] - ETA: 3:24 - loss: 0.4222 - acc: 0.8510

...

50000/50000 [==============================] - 330s 7ms/step - loss: 0.0969 - acc: 0.9642 - val_loss: 1.0558 - val_acc: 0.8163

Epoch 00042: val_acc did not improve from 0.86000
Epoch 43/50
50000/50000 [==============================] - 329s 7ms/step - loss: 0.0919 - acc: 0.9670 - val_loss: 0.6695 - val_acc: 0.8582

Epoch 00043: val_acc did not improve from 0.86000
Epoch 44/50
18944/50000 [==========>...................] - ETA: 3:09 - loss: 0.0906 - acc: 0.9681

50000/50000 [==============================] - 330s 7ms/step - loss: 0.0982 - acc: 0.9645 - val_loss: 0.6196 - val_acc: 0.8725

Epoch 00044: val_acc improved from 0.86000 to 0.87250, saving model to weights.best.hdf5
Epoch 45/50
50000/50000 [==============================] - 330s 7ms/step - loss: 0.0855 - acc: 0.9683 - val_loss: 0.6981 - val_acc: 0.8623

Epoch 00045: val_acc did not improve from 0.87250
Epoch 46/50
13568/50000 [=======>......................] - ETA: 3:42 - loss: 0.0844 - acc: 0.9713

50000/50000 [==============================] - 330s 7ms/step - loss: 0.0881 - acc: 0.9689 - val_loss: 0.9814 - val_acc: 0.8112

Epoch 00046: val_acc did not improve from 0.87250
Epoch 47/50
50000/50000 [==============================] - 329s 7ms/step - loss: 0.0906 - acc: 0.9674 - val_loss: 0.6442 - val_acc: 0.8620

Epoch 00047: val_acc did not improve from 0.87250
Epoch 48/50
18176/50000 [=========>....................] - ETA: 3:13 - loss: 0.0735 - acc: 0.9733

50000/50000 [==============================] - 329s 7ms/step - loss: 0.0821 - acc: 0.9705 - val_loss: 0.9226 - val_acc: 0.8419

Epoch 00048: val_acc did not improve from 0.87250
Epoch 49/50
50000/50000 [==============================] - 331s 7ms/step - loss: 0.0799 - acc: 0.9713 - val_loss: 0.9739 - val_acc: 0.8238

Epoch 00049: val_acc did not improve from 0.87250
Epoch 50/50
18944/50000 [==========>...................] - ETA: 3:09 - loss: 0.0754 - acc: 0.9740

50000/50000 [==============================] - 330s 7ms/step - loss: 0.0741 - acc: 0.9742 - val_loss: 0.7733 - val_acc: 0.8559

Epoch 00050: val_acc did not improve from 0.87250


In [0]:
# load weights
model.load_weights("weights.best.hdf5")



In [29]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 33s 3ms/step
Test loss: 0.619635308033228
Test accuracy: 0.8725


In [30]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')